## Import Libraries and Setup Argoverse Stereo Data Loader 

In [1]:
%matplotlib notebook

import copy
import json
import shutil
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import pandas as pd

from argoverse.data_loading.stereo_dataloader import ArgoverseStereoDataLoader
from argoverse.evaluation.stereo.eval import StereoEvaluator
from argoverse.utils.calibration import get_calibration_config
from argoverse.utils.camera_stats import RECTIFIED_STEREO_CAMERA_LIST
import os
from multiprocessing import Pool
import time

STEREO_FRONT_LEFT_RECT = RECTIFIED_STEREO_CAMERA_LIST[0]
STEREO_FRONT_RIGHT_RECT = RECTIFIED_STEREO_CAMERA_LIST[1]


# Path to the dataset (please change accordingly).
data_dir = "/home/cmiller/Documents/Curtis_Classes/CMPE_249/Project/code/argoverse_stereo_v1.1/"

# Choosing the data split: train, val, or test (note that we do not provide ground truth for the test set).
split_name = "val"

# Choosing a specific log id. For example, 273c1883-673a-36bf-b124-88311b1a80be.
log_ids = os.listdir('./argoverse_stereo_v1.1/rectified_stereo_images_v1.1/val/')

# Creating the Argoverse Stereo data loader.
stereo_data_loader = ArgoverseStereoDataLoader(data_dir, split_name)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define Function to Create Disparity Map 

In [2]:
def process_stereo_pair(stereo_data_loader, left_stereo_img_fpaths, right_stereo_img_fpaths,
                        disparity_map_fpaths, disparity_obj_map_fpaths, idx, log_id, resize,
                        save_path):
    
    # Loading the rectified stereo images.
    stereo_front_left_rect_image = stereo_data_loader.get_rectified_stereo_image(left_stereo_img_fpaths[idx])
    stereo_front_right_rect_image = stereo_data_loader.get_rectified_stereo_image(right_stereo_img_fpaths[idx])
    
    # Scale image down
    if resize:
        width = int(stereo_front_left_rect_image.shape[1] * 0.25)
        height = int(stereo_front_left_rect_image.shape[0] * 0.25)
        dim = (width, height)
        stereo_front_left_rect_image = cv2.resize(stereo_front_left_rect_image, dim)
        stereo_front_right_rect_image = cv2.resize(stereo_front_right_rect_image, dim)
    
    # Defining the SGM parameters (please check the OpenCV documentation for details).
    # We found this parameters empirically and based on the Argoverse Stereo data.
    max_disp = 192
    if resize:
        max_disp = int(max_disp/4)
    win_size = 5
    uniqueness_ratio = 100
    speckle_window_size = 200
    speckle_range = 2
    block_size = 2
    P1 = int(24 * win_size ** 2)
    P2 = int(96 * win_size ** 2)
    preFilterCap = 63 

    # Defining the Weighted Least Squares (WLS) filter parameters.
    lmbda = 8000.0
    sigma = 2

    import time
    t0 = time.time()
    # Defining the SGM left matcher.
    left_matcher = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=max_disp,
        blockSize=block_size,
        P1=P1,
        P2=P2,
        disp12MaxDiff=1,
        uniquenessRatio=uniqueness_ratio,
        speckleWindowSize=speckle_window_size,
        speckleRange=speckle_range,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY,
        preFilterCap=preFilterCap,
    )

    # Defining the SGM right matcher needed for the left-right consistency check in the WLS filter.
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    # Defining the WLS filter.
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)

    # Computing the disparity maps.
    left_disparity = left_matcher.compute(stereo_front_left_rect_image, stereo_front_right_rect_image)
    right_disparity = right_matcher.compute(stereo_front_right_rect_image, stereo_front_left_rect_image)

    # Applying the WLS filter.
    left_disparity_pred = wls_filter.filter(left_disparity, stereo_front_left_rect_image, None, right_disparity)

    # Recovering the disparity map.
    # OpenCV produces a disparity map as a signed short obtained by multiplying subpixel shifts with 16.
    # To recover the true disparity values, we need to divide the output by 16 and convert to float.
    left_disparity_pred = np.float32(left_disparity_pred) / 16.0

    # OpenCV will also set negative values for invalid disparities where matches could not be found.
    # Here we set all invalid disparities to zero.
    left_disparity_pred[left_disparity_pred < 0] = 0
    
    # Encoding the real disparity values to an uint16 data format to save as an uint16 PNG file.
    if resize:
        left_disparity_pred = np.uint16(left_disparity_pred * 4 * 256.0)
        width = int(stereo_front_left_rect_image.shape[1] * 4)
        height = int(stereo_front_left_rect_image.shape[0] * 4)
        dim = (width, height)
        left_disparity_pred = cv2.resize(left_disparity_pred, dim)
        left_disparity_pred[:,0:194] = 0
    else:
        left_disparity_pred = np.uint16(left_disparity_pred * 256.0)
    timestamp = int(Path(disparity_map_fpaths[idx]).stem.split("_")[-1])

    # Change the path to the directory you would like to save the result.
    # The log id must be consistent with the stereo images' log id.
    save_dir_disp = f"{save_path}/{log_id}/"
    Path(save_dir_disp).mkdir(parents=True, exist_ok=True)

    # The predicted disparity filename must have the format: 'disparity_[TIMESTAMP OF THE LEFT STEREO IMAGE].png' 
    filename = f"{save_dir_disp}/disparity_{timestamp}.png"

    # Writing the PNG file to disk.
    cv2.imwrite(filename, left_disparity_pred)
                        

## Produce Disparity Maps from Full Scale Images 

In [3]:
t0 = time.time()
imageCount = 0
for log_id in log_ids:
    
    # Loading the left rectified stereo image paths for the chosen log.
    left_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id,
        camera_name=STEREO_FRONT_LEFT_RECT)
    
    # Loading the right rectified stereo image paths for the chosen log.
    right_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id,
        camera_name=STEREO_FRONT_RIGHT_RECT)
    
    # Loading the disparity map paths for the chosen log.
    disparity_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_disparity")

    # Loading the disparity map paths for foreground objects for the chosen log.
    disparity_obj_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_objects_disparity")
    
    args = [(stereo_data_loader, left_stereo_img_fpaths, right_stereo_img_fpaths,
             disparity_map_fpaths, disparity_obj_map_fpaths, idx, log_id, False,
             './results/stereo_match_results')
             for idx in range(0,len(left_stereo_img_fpaths))]
    
    num_procs = 8
    with Pool(num_procs) as p:
        p.starmap(process_stereo_pair, args)
        
    imageCount = imageCount + len(left_stereo_img_fpaths) 
    print('Elapsed Time:', time.time()-t0, 'Time per Image:', (time.time()-t0)/imageCount)

Elapsed Time: 262.69988441467285 Time per Image: 3.6486095355616674
Elapsed Time: 528.1798450946808 Time per Image: 3.617670180046395
Elapsed Time: 910.2800421714783 Time per Image: 3.5979448354291352
Elapsed Time: 1175.7594847679138 Time per Image: 3.5955947582874823
Elapsed Time: 1419.9189069271088 Time per Image: 3.5947314129600043
Elapsed Time: 1682.7530355453491 Time per Image: 3.5956261473843174
Elapsed Time: 1947.9567239284515 Time per Image: 3.5940160980083844
Elapsed Time: 2213.0128180980682 Time per Image: 3.5925532780684435
Elapsed Time: 2729.4358353614807 Time per Image: 3.5960946488286196
Elapsed Time: 2992.6756315231323 Time per Image: 3.5969659047057996
Elapsed Time: 3366.3946709632874 Time per Image: 3.5927371106000185
Elapsed Time: 3632.433248758316 Time per Image: 3.5964685645433936
Elapsed Time: 4163.276529788971 Time per Image: 3.6014502865220437
Elapsed Time: 4428.519716262817 Time per Image: 3.600422535097696
Elapsed Time: 4694.778831243515 Time per Image: 3.60305

## Produce Disparity Maps from Downsampled Images 

In [4]:
t0 = time.time()
imageCount = 0
for log_id in log_ids:
    
    # Loading the left rectified stereo image paths for the chosen log.
    left_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id,
        camera_name=STEREO_FRONT_LEFT_RECT)
    
    # Loading the right rectified stereo image paths for the chosen log.
    right_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id,
        camera_name=STEREO_FRONT_RIGHT_RECT)
    
    # Loading the disparity map paths for the chosen log.
    disparity_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_disparity")

    # Loading the disparity map paths for foreground objects for the chosen log.
    disparity_obj_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_objects_disparity")
    
    args = [(stereo_data_loader, left_stereo_img_fpaths, right_stereo_img_fpaths,
             disparity_map_fpaths, disparity_obj_map_fpaths, idx, log_id, True,
            './results/stereo_match_downsampled_results')
             for idx in range(0,len(left_stereo_img_fpaths))]
    
    num_procs = 8
    with Pool(num_procs) as p:
        p.starmap(process_stereo_pair, args)
        
    imageCount = imageCount + len(left_stereo_img_fpaths) 
    print('Elapsed Time:', time.time()-t0, 'Time per Image:', (time.time()-t0)/imageCount)

Elapsed Time: 4.9936301708221436 Time per Image: 0.06935603419939677
Elapsed Time: 10.461380243301392 Time per Image: 0.07165332199776009
Elapsed Time: 19.02753734588623 Time per Image: 0.07520767068674442
Elapsed Time: 24.58074688911438 Time per Image: 0.07517048999074767
Elapsed Time: 29.70619559288025 Time per Image: 0.07520556570608404
Elapsed Time: 35.17054843902588 Time per Image: 0.07515075013168857
Elapsed Time: 40.86081528663635 Time per Image: 0.07538896469172517
Elapsed Time: 46.73406195640564 Time per Image: 0.07586698795293832
Elapsed Time: 58.38537883758545 Time per Image: 0.07692408655943135
Elapsed Time: 64.07482886314392 Time per Image: 0.07701302119172536
Elapsed Time: 72.69126868247986 Time per Image: 0.07757873316330233
Elapsed Time: 79.31254076957703 Time per Image: 0.07852727162956011
Elapsed Time: 91.00778222084045 Time per Image: 0.07872645850825062
Elapsed Time: 97.27216625213623 Time per Image: 0.07908306528882282
Elapsed Time: 103.13731646537781 Time per Imag

## Get Performance Results from Disparity Map Results 

In [7]:
t0 = time.time()
imageCount = 0
data = pd.DataFrame()

for log_id in log_ids:
    # Path to the predicted disparity maps.
    save_dir_disp = f'./results/stereo_match_results/{log_id}/'
    pred_dir = Path(save_dir_disp)

    # Path to the ground-truth disparity maps.
    gt_dir = Path(f"{data_dir}/disparity_maps_v1.1/{split_name}/{log_id}")

    # Path to save the disparity error image.
    save_figures_dir = Path(f'./results/stereo_match_error_results/')
    save_figures_dir.mkdir(parents=True, exist_ok=True)

    # Creating the stereo evaluator.
    evaluator = StereoEvaluator(
        pred_dir,
        gt_dir,
        save_figures_dir,
        save_disparity_error_image=True,
        num_procs=-1,
    )

    # Running the stereo evaluation.
    metrics, data, errors = evaluator.evaluate(data)
    print('Elapsed Time:', time.time()-t0)
    
# Printing the quantitative results (using json trick for organized printing).
print('Results for Full Images')
print('==============================')
print(f"{json.dumps(metrics, sort_keys=False, indent=4)}")

Elapsed Time: 24.07984685897827
Elapsed Time: 48.77725100517273
Elapsed Time: 81.85719347000122
Elapsed Time: 106.3549792766571
Elapsed Time: 128.96713423728943
Elapsed Time: 153.00676012039185
Elapsed Time: 178.38883304595947
Elapsed Time: 203.52759218215942
Elapsed Time: 246.55882859230042
Elapsed Time: 270.4816973209381
Elapsed Time: 303.53218173980713
Elapsed Time: 328.23563146591187
Elapsed Time: 374.5974049568176
Elapsed Time: 400.16751408576965
Elapsed Time: 424.9032804965973
Elapsed Time: 469.76299715042114
Elapsed Time: 495.6985549926758
Results for Full Images
{
    "all:10": 9.950958264474124,
    "fg:10": 17.703895996797574,
    "bg:10": 7.939138760765811,
    "all*:10": 8.677172032394893,
    "fg*:10": 15.07139168968646,
    "bg*:10": 7.0176428261202,
    "all:5": 17.368416343031573,
    "fg:5": 21.961189501407933,
    "bg:5": 16.17663189497558,
    "all*:5": 15.64928336175325,
    "fg*:5": 19.00932280991411,
    "bg*:5": 14.777232682027314,
    "all:3": 21.779042072217564

In [6]:
t0 = time.time()
imageCount = 0
data = pd.DataFrame()

for log_id in log_ids:
    # Path to the predicted disparity maps.
    save_dir_disp = f'./results/stereo_match_downsampled_results/{log_id}/'
    pred_dir = Path(save_dir_disp)

    # Path to the ground-truth disparity maps.
    gt_dir = Path(f"{data_dir}/disparity_maps_v1.1/{split_name}/{log_id}")

    # Path to save the disparity error image.
    save_figures_dir = Path(f'./results/stereo_match_downsampled_error_results/')
    save_figures_dir.mkdir(parents=True, exist_ok=True)

    # Creating the stereo evaluator.
    evaluator = StereoEvaluator(
        pred_dir,
        gt_dir,
        save_figures_dir,
        save_disparity_error_image=True,
        num_procs=-1,
    )

    # Running the stereo evaluation.
    metrics, data, errors = evaluator.evaluate(data)
    print('Elapsed Time:', time.time()-t0)
    
# Printing the quantitative results (using json trick for organized printing).
print('Results for Downsampled Images')
print('==============================')
print(f"{json.dumps(metrics, sort_keys=False, indent=4)}")

Elapsed Time: 23.896005630493164
Elapsed Time: 48.65390062332153
Elapsed Time: 82.03866076469421
Elapsed Time: 106.62121963500977
Elapsed Time: 128.7276966571808
Elapsed Time: 152.5318911075592
Elapsed Time: 177.43518137931824
Elapsed Time: 202.87696981430054
Elapsed Time: 247.54599285125732
Elapsed Time: 272.29368233680725
Elapsed Time: 305.6698877811432
Elapsed Time: 329.20626759529114
Elapsed Time: 376.1516020298004
Elapsed Time: 401.53325176239014
Elapsed Time: 426.2269606590271
Elapsed Time: 471.0610213279724
Elapsed Time: 496.22927618026733
Results for Downsampled Images
{
    "all:10": 10.512892162635849,
    "fg:10": 15.873847840167404,
    "bg:10": 9.121771072418912,
    "all*:10": 9.343688891046305,
    "fg*:10": 13.155248967380102,
    "bg*:10": 8.354962002717825,
    "all:5": 19.490086525233316,
    "fg:5": 20.20260489056951,
    "bg:5": 19.305194243929755,
    "all*:5": 18.051210467691185,
    "fg*:5": 17.191842430319344,
    "bg*:5": 18.2741323864077,
    "all:3": 24.3115